In [293]:
%run 01_hash.ipynb

In [294]:
from copy import deepcopy

Print objects in a nice way.

In [295]:
def stringfy(o):
    s = []
    for k,v in o.__dict__.items():
        p = f'{k}:'.ljust(15)
        if hasattr(v,'messageHash'): s.append(p+ph(v.messageHash.hex()))
        elif k == 'time'           : s.append(p+time.ctime(v))
        elif str(v)[:2]=='0x'      : s.append(p+ph(v))
        elif type(v)   ==float     : s.append(p+str(round(v,8))+' ether')
        else                       : s.append(p+str(v))
    return '\n'.join(s)

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [297]:
class TX(Hashable): 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = float(value)
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.time()
        self.signed      = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop not in ['sig','signed']: super().__setattr__('hash', sha(self.__dict__))
        if prop == 'sig'               : self.signed = True
    
    def __str__(self): return stringfy(self)

Create random txs.

In [298]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0)

In [287]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

fr:            📔 0x6e3fee8cf0...626
hash:          👹 0x13f5d352f2...7ac
to:            📂 0x5c6b8b588a...702
value:         12.0 ether
fee:           0.2 ether
nonce:         0
time:          Tue Mar 30 21:03:52 2021
signed:        False

fr:            💒 0x2cba7c15b1...3f9
hash:          👷 0x11e37030fe...7d9
to:            🕓 0xedc3eb982a...82d
value:         6.0 ether
fee:           0.15 ether
nonce:         0
time:          Tue Mar 30 21:03:52 2021
signed:        False



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [288]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

fr:            💒 0x2cba7c15b1...3f9
hash:          📳 0x8d06f1048d...51a
to:            🕓 0xedc3eb982a...82d
value:         120
fee:           0.15 ether
nonce:         0
time:          Tue Mar 30 21:03:52 2021
signed:        False


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [289]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [290]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

fr:            📔 0x6e3fee8cf0...626
hash:          👹 0x13f5d352f2...7ac
to:            📂 0x5c6b8b588a...702
value:         12.0 ether
fee:           0.2 ether
nonce:         0
time:          Tue Mar 30 21:03:52 2021
signed:        True
sig:           📾 0x9849997682...1a7


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [291]:
assert tx1 == tx1_signed